# Yelp Restaurant Scraping

In [42]:
import pandas as pd
import numpy as np

## Upload NYC Open Data Restaurants Inspection Results

NYC Restaurant Inspection Data from following website:
https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j

In [43]:
restaurants_df = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv', encoding='UTF-8')

In [44]:
# Glimpse of the raw data
restaurants_df.head(2)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,40545130,SPEEDY'S DELI,MANHATTAN,1271,BROADWAY,10001.0,2126838997,American,07/16/2018,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,12.0,A,07/16/2018,11/21/2018,Cycle Inspection / Re-inspection
1,50001204,MON CHER MARKET,MANHATTAN,339,BROADWAY,10013.0,2129650007,American,04/06/2018,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,21.0,NaN,NaN,11/21/2018,Cycle Inspection / Initial Inspection


In [45]:
# Clean data by removing duplicate CAMIS Records (keep most recent), removing restaurants without a valid rating, and removing restaurants located in Queens.
restaurants_df = restaurants_df[restaurants_df.BORO != 'QUEENS'].sort_values(['CAMIS', 'INSPECTION DATE'], ascending=[True, False]).drop_duplicates(subset=['CAMIS'], keep='first').dropna(subset=['GRADE'])

In [46]:
# Define a helper method that obtains the Yelp URL for a given restuarant name and borough
def get_url(df):
    url = str(df['DBA']).replace(' ','-')
    if '&' in url:
        url = url.replace('&', 'AND')
    if '\'' in url:
        url = url.replace('\'', '')
    if df['BORO'] == 'MANHATTAN':
        url += '-new-york'
    else:
        url += "-" + str(df['BORO']).replace(" ", "-")
    url = "https://www.yelp.com/biz/" + url
    return url

In [47]:
# Apply the get_url function on the dataframe and add as a new column
restaurants_df['URL'] = restaurants_df.apply(get_url, axis = 1)

## Scrape Yelp data for reviews

In [11]:
# Make a copy of the first 10 restaurants to test
test_df = restaurants_df.head(10)
test_df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,URL
91711,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,05/18/2017,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,7.0,A,05/18/2017,11/21/2018,Cycle Inspection / Initial Inspection,https://www.yelp.com/biz/MORRIS-PARK-BAKE-SHOP...
151360,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,10/27/2016,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,Critical,11.0,A,10/27/2016,11/21/2018,Cycle Inspection / Re-inspection,https://www.yelp.com/biz/WENDYS-BROOKLYN
120414,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,12/14/2017,Violations were cited in the following area(s).,04H,"Raw, cooked or prepared food is adulterated, c...",Critical,10.0,A,12/14/2017,11/21/2018,Cycle Inspection / Re-inspection,https://www.yelp.com/biz/DJ-REYNOLDS-PUB-AND-R...
300624,40356018,RIVIERA CATERERS,BROOKLYN,2780,STILLWELL AVENUE,11224.0,7183723031,American,10/30/2018,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,10.0,A,10/30/2018,11/21/2018,Cycle Inspection / Initial Inspection,https://www.yelp.com/biz/RIVIERA-CATERERS-BROO...
8840,40356483,WILKEN'S FINE FOOD,BROOKLYN,7114,AVENUE U,11234.0,7184443838,Delicatessen,06/03/2017,Violations were cited in the following area(s).,09A,Canned food product observed dented and not se...,Not Critical,13.0,A,06/03/2017,11/21/2018,Cycle Inspection / Initial Inspection,https://www.yelp.com/biz/WILKENS-FINE-FOOD-BRO...
126087,40356731,TASTE THE TROPICS ICE CREAM,BROOKLYN,1839,NOSTRAND AVENUE,11226.0,7188560821,"Ice Cream, Gelato, Yogurt, Ices",06/26/2015,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,5.0,A,06/26/2015,11/21/2018,Cycle Inspection / Initial Inspection,https://www.yelp.com/biz/TASTE-THE-TROPICS-ICE...
22234,40357217,WILD ASIA,BRONX,2300,SOUTHERN BOULEVARD,10460.0,7182207846,American,06/14/2017,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2.0,A,06/14/2017,11/21/2018,Cycle Inspection / Initial Inspection,https://www.yelp.com/biz/WILD-ASIA-BRONX
72155,40359480,1 EAST 66TH STREET KITCHEN,MANHATTAN,1,EAST 66 STREET,10065.0,2128793900,American,09/28/2018,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,7.0,A,09/28/2018,11/21/2018,Cycle Inspection / Initial Inspection,https://www.yelp.com/biz/1-EAST-66TH-STREET-KI...
72110,40359705,NATHAN'S FAMOUS,BROOKLYN,1310,SURF AVENUE,11224.0,7183332202,Hotdogs,09/08/2015,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,11.0,A,09/08/2015,11/21/2018,Cycle Inspection / Re-inspection,https://www.yelp.com/biz/NATHANS-FAMOUS-BROOKLYN
22959,40360045,SEUDA FOODS,BROOKLYN,705,KINGS HIGHWAY,11223.0,7183751500,Jewish/Kosher,11/27/2017,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,13.0,A,11/27/2017,11/21/2018,Cycle Inspection / Initial Inspection,https://www.yelp.com/biz/SEUDA-FOODS-BROOKLYN


In [48]:
from bs4 import BeautifulSoup
from requests import get 
import re

In [49]:
def scrape_reviews(df):
    response = get(df['URL'])
    
    html_soup = BeautifulSoup(response.text, "html.parser")
    review_containers = html_soup.find_all("div", class_ = "review review--with-sidebar")
    if len(review_containers) == 0:
        return pd.DataFrame()
    
    review_list = []
    review_list.append(df['CAMIS'])

    for i in range(len(review_containers)):
        review = str((review_containers[i]).p)
        clean = re.compile('<.*?>')
        review = re.sub(clean, '', review)
        review_list.append(review)

    new_df = pd.DataFrame(review_list)

    return new_df

In [50]:
# Define new reviews_df to store 20 most recent Yelp Reviews
reviews_df = pd.DataFrame()

for index, row in restaurants_df.iterrows():
    res = scrape_reviews(row)
    reviews_df = reviews_df.append(res.T)

SSLError: HTTPSConnectionPool(host='www.yelp.com', port=443): Max retries exceeded with url: /biz/LA-VECINA-RESTAURANT-BRONX (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')")))

In [51]:
reviews_df = pd.DataFrame(reviews_df)
reviews_df = reviews_df.set_index(0)

In [52]:
reviews_df.index.names = ['CAMIS']

In [53]:
reviews_df.to_csv("Reviews.csv")

In [54]:
restaurants_df.to_csv("Restaurants.csv")